In [1]:
import pandas as pd
import numpy as np

# Calculating sampling target sizes

**Step 1:** With the population counts aggregated in the previous step, we will need to calculate the sample size need to achieve adequite statistical power, using experience of harrassment as the critical variable. We will be using [Cochran's formula](https://www.tarleton.edu/academicassessment/documents/samplesize.pdf) to calculate appropriate sample sizes:

\begin{equation}
n_0=\frac{Z^2pq}{e^2}
\end{equation}
<div style="padding-left: 40%; font-size:12px">
where:<br>
    p = 0.215 (incidence of critical variable)<br>
    q = 1 - p<br>
    e = 0.05 (margin of error)<br>
    Z<sup>2</sup> = 1.96 (for 95% confidence level)
</div><br>

**Step 2:** n<sub>0</sub> needs to be further modified for each project to adjust the sample to the project's population, referred to as Finite Population Correction For Proportions, can achieved using the following equation:

\begin{equation}
n = \frac{n_0}{1 + \frac{(n_0 - 1)}{N}}
\end{equation}
<div style="padding-left: 40%; font-size:12px">
where: N = population size of a strata
</div><br>

**Step 3:** The sample size caclulated for each projects needs to be divided across various editing activity levels. This will be according to the proportions of the population. For example, if the desired sample size for arwiki is 117, and in the population, if 10% of the editors belong to 10-29 strata, the sample size for 10-29 strata on arwiki would be 12.

**Step 4:** Based on previous year's response rates, the sample size needs needs to be adjusted to account for nonresponse. This is capped at strata-level population). For example, 49 of the most-active (1200+) editors on en.wikipedia, and the 2020 response rate was 29%, we'd expect to need to contact 170 such contributors to achieve our target. 

### Step 1: Calculating n<sub>0</sub>

In [2]:
p = 0.215 # indence of critical variable (% experience harrasment)
e = 0.05 # margin of error
Z_square = 1.96 # 95% confidence

nO = round((Z_square**2 * p * (1-p))/e**2) #Cochran's formula
print(nO)

259


### Step 2: Adjusting the sample to project's population

In [3]:
# dataframe with population sizes for each strata
strataN = pd.read_csv('definitions/strata-populations.tsv', sep='\t')
strataN['total'] = strataN[strataN.columns[1:]].sum(axis=1)
strataN.head()

,project_group,10-29,30-149,150-599,600-1199,1200+,total
0,arwiki,18,53,59,29,109,268
1,asia_wps,12,49,75,54,126,316
2,cee_wps,137,512,631,318,1000,2598
3,commons,116,422,502,307,1479,2826
4,dewiki,93,599,715,351,869,2627


In [4]:
project_target_sizes = pd.DataFrame([strataN['project_group'],
                                     strataN['total']
                                     .apply(lambda N: round(nO/(1+(nO-1)/N)))]).transpose()
project_target_sizes.head()

,project_group,total
0,arwiki,132
1,asia_wps,143
2,cee_wps,236
3,commons,237
4,dewiki,236


### Step 3: Dividing the sample size across various edit bins

In [5]:
# dataframe with percentage distribution of 
bins_labels = strataN.columns[1:-1]
strataN_perc = strataN.copy()
strataN_perc[bins_labels] = (strataN_perc[bins_labels]
                             .div(strataN_perc[bins_labels].sum(axis=1), axis=0)
                             .multiply(100).round(2))
strataN_perc.drop('total', axis=1, inplace=True)
strataN_perc.head()

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,6.72,19.78,22.01,10.82,40.67
1,asia_wps,3.80,15.51,23.73,17.09,39.87
2,cee_wps,5.27,19.71,24.29,12.24,38.49
3,commons,4.10,14.93,17.76,10.86,52.34
4,dewiki,3.54,22.80,27.22,13.36,33.08


In [6]:
# sampling target sizes by each strata (projects and edit bins)
strata_target_sizes = strataN_perc.copy()
strata_target_sizes[bins_labels] = (strata_target_sizes[bins_labels].multiply(0.01)
                                    .mul(project_target_sizes.total.values, axis=0)
                                    .astype(float).round())
strata_target_sizes.head()

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,9.0,26.0,29.0,14.0,54.0
1,asia_wps,5.0,22.0,34.0,24.0,57.0
2,cee_wps,12.0,47.0,57.0,29.0,91.0
3,commons,10.0,35.0,42.0,26.0,124.0
4,dewiki,8.0,54.0,64.0,32.0,78.0


### Step 4: Adjust sampling target sizes to account for non-response

In [7]:
# response rates for each strata; provided GDI based on the previous cycle of the survey
strata_resp_rates = pd.read_csv('secrets/strata-response-rates.tsv', sep='\t')
strata_resp_rates.head()

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,0.08,0.04,0.03,0.10,0.04
1,asia_wps,0.00,0.00,0.08,0.09,0.09
2,cee_wps,0.02,0.03,0.05,0.05,0.06
3,commons,0.03,0.07,0.08,0.10,0.09
4,dewiki,0.11,0.10,0.10,0.10,0.12


In [8]:
strata_sample_pull = strata_resp_rates.copy()

for index in strata_sample_pull.index:
    for e_bin in bins_labels:
        # if the response rate is 0; then the population is multiplied by 0.057 to get the target
        if strata_resp_rates.loc[index, e_bin] == 0:
            strata_target = round(strataN.loc[index, 'total']*0.057)
            
            # if a strata's target size is greater than the population size
            # then the target size is capped at population size
            if strata_target > strataN.loc[index, e_bin]:
                strata_sample_pull.loc[index, e_bin] = strataN.loc[index, e_bin]
            else:
                strata_sample_pull.loc[index, e_bin] = strata_target
        
        else:
            # adjusting the sampling target according the response rate
            strata_target = strata_target_sizes.loc[index, e_bin]/strata_resp_rates.loc[index, e_bin]
            
            # if a strata's target size is greater than the population size
            # then the target size is capped at population size            
            if  strata_target > strataN.loc[index, e_bin]:
                strata_sample_pull.loc[index, e_bin] = strataN.loc[index, e_bin]
            else:
                strata_sample_pull.loc[index, e_bin] = strata_target
                
strata_sample_pull[bins_labels] = strata_sample_pull[bins_labels].astype(float).round()
strata_sample_pull

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,18.0,53.0,59.0,29.0,109.0
1,asia_wps,12.0,18.0,75.0,54.0,126.0
2,cee_wps,137.0,512.0,631.0,318.0,1000.0
3,commons,116.0,422.0,502.0,260.0,1378.0
4,dewiki,73.0,540.0,640.0,320.0,650.0
5,enwiki,544.0,1480.0,850.0,400.0,636.0
6,eswiki,75.0,309.0,363.0,161.0,364.0
7,frwiki,127.0,568.0,549.0,235.0,653.0
8,itwiki,45.0,227.0,267.0,107.0,324.0
9,jawiki,103.0,499.0,441.0,135.0,239.0


In [9]:
strata_sample_pull.to_csv('definitions/strata-sample-targets.tsv', sep='\t')